In [1]:
import bs4
from typing import List
import requests
import os
import tqdm

In [2]:
def get_title_and_episode_map(url:str)-> List[str]:
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    title = soup.find('h1',{'class':'subj'}).text.strip()
    
    content = soup.find('ul',{'id':'_listUl'})
    items = content.find_all('li')
    result = {}
    
    for i in items:
        url = i.a['href']
        name = i.attrs['data-episode-no']
        result[name] = url
    return title , result
    

In [3]:
def download_manga(url:str) -> None:
    title, episode_map = get_title_and_episode_map(url)
    os.mkdir(title)
    for name,url in tqdm.tqdm(episode_map.items()):
        
        path = os.path.join(title,name)
        os.mkdir(path)
        images = get_images(url)
        download_image(images,path)

In [4]:
def get_images(url:str)->List[str]:
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    
    content = soup.find('div',{'id':'_imageList'})
    images = content.find_all('img')
    result = []
    for i in images:
        result.append(i.attrs['data-url'])
    return result

In [5]:
def download_image(image_list,path)->None:
    
    for index,image in enumerate(image_list):
        
        resp = requests.get(image,headers={'referer':'https://www.webtoons.com'})
        
        with open(os.path.join(path,f'{str(index)}.png',),'wb') as img:
            img.write(resp.content)

In [6]:
def get_daily_urls(url,day='MONDAY'):
    result = []
    r = soup.find('div',{'class',f'_list_{day}'})
    items = r.find_all('li')
    for i in items:
        title = i.find('p',{'class','subj'}).text
        episode_url = i.a['href']
        result.append(episode_url)
    return result

In [23]:
daily_urls = get_daily_urls('https://www.webtoons.com/en/dailySchedule',day='FRIDAY')

In [ ]:
for url in daily_urls:
    download_manga(url)

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [09:44<02:48, 84.02s/it]

In [7]:
get_images('https://www.webtoons.com/en/fantasy/tower-of-god/season-1-ep-0/viewer?title_no=95&episode_no=1')

['https://webtoon-phinf.pstatic.net/20200328_249/1585334566015rEpaA_JPEG/15853345629669517.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_277/1585334566029jg5xc_JPEG/15853345634229512.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_202/1585334566015OIWyG_JPEG/15853345629669514.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_228/1585334566028ThPFr_JPEG/15853345628509518.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_65/158533456601392pEY_JPEG/15853345629489519.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_37/1585334568147mJ6lL_JPEG/15853345681279514.jpg?type=q90',
 'https://webtoon-phinf.pstatic.net/20200328_73/1585334568322LdmJh_JPEG/15853345682999517.jpg?type=q90']

In [9]:
url = 'https://www.webtoons.com/en/fantasy/tower-of-god/season-1-ep-0/viewer?title_no=95&episode_no=1'
resp = requests.get(url)
soup = bs4.BeautifulSoup(resp.content,'html.parser')
p = soup.find('div',{'class':'paginate'})
p.a['href']

'https://www.webtoons.com/en/fantasy/tower-of-god/season-1-ep-1-1fheadons-floor/viewer?title_no=95&episode_no=2'

In [10]:
url = 'https://www.webtoons.com/en/fantasy/tower-of-god/season-1-ep-0/viewer?title_no=95&episode_no=1'

In [28]:
def get_from_first(url:str)-> None:
    
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.content,'html.parser')
    name = soup.find('h1',{'class':'subj_episode'}).text.strip()
    
    images = get_images(url)
    os.mkdir(name)
    download_image(images,name)
    
    next_episode = soup.find('a',{'class':'pg_next'})['href']
    
    while next_episode:
        

        resp = requests.get(next_episode)
        soup = bs4.BeautifulSoup(resp.content,'html.parser')

        name = soup.find('h1',{'class':'subj_episode'}).text.strip()
        images = get_images(next_episode)
        os.mkdir(name)
        download_image(images,name)
        
        next_episode = soup.find('a',{'class':'pg_next'})['href']
        
        print(next_episode)
    

In [30]:
get_from_first('https://www.webtoons.com/en/super-hero/unordinary/prologue/viewer?title_no=679&episode_no=1')

https://www.webtoons.com/en/super-hero/unordinary/episode-2/viewer?title_no=679&episode_no=3
https://www.webtoons.com/en/super-hero/unordinary/episode-3/viewer?title_no=679&episode_no=4
https://www.webtoons.com/en/super-hero/unordinary/episode-4/viewer?title_no=679&episode_no=5
https://www.webtoons.com/en/super-hero/unordinary/episode-5/viewer?title_no=679&episode_no=6


KeyboardInterrupt: 